In [25]:
%matplotlib inline
from math import *
import numpy as Numpy
#import utils
import matplotlib.pyplot as Matplotlib
import copy
import pandas as pd
import time
import datetime
import resource 

import warnings
warnings.filterwarnings('ignore')

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
PORTFOLIO = pd.read_excel(r'Programming_Exercise_Extra/SamplePortfolio.xlsx', sheet_name='Portfolio')
display(PORTFOLIO)
IRCURVES = pd.read_excel(r'Programming_Exercise_Extra/SamplePortfolio.xlsx', sheet_name='IR Curves')
display(IRCURVES)
PSHOCKS = pd.read_excel(r'Programming_Exercise_Extra/SamplePortfolio.xlsx', sheet_name='Partial shocks')
display(PSHOCKS)
CTABLE = pd.read_excel(r'Programming_Exercise_Extra/SamplePortfolio.xlsx', sheet_name='Correlation table')
display(CTABLE)

,Valuation date,Maturity,Direction,Asset type,Asset ID,Option type,Quote ccy,Portfolio ccy,Mkt val qccy,Mkt val pccy,...,Cpn type,Cpn freq,Next cpn,Grouping ID 2 legs,FX up,FX down,IR up,IR down,EQ down,Spread up
0,2019-12-31,2020-03-31,S,Equity options,SX5E Index 03/31 P 3700,P,EUR,USD,-2.467432e+04,-27701.877615,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-31,2025-04-15,L,Bonds,TNOTE,NaN,USD,USD,1.010570e+05,101057.000000,...,Fixed,2.0,2020-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-31,2020-09-30,S,FX forward,FWD B:USD S:JPY 106 09/30,NaN,JPY,USD,-1.000638e+07,-92072.403261,...,NaN,NaN,NaT,FWDJPY0930,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-31,2021-06-30,L,Bonds,UKST,NaN,GBP,USD,2.021140e+05,268155.071870,...,Floating,4.0,2020-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-31,NaT,L,Stocks,SX5E,NaN,EUR,USD,1.124541e+05,126252.302618,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-12-31,NaT,L,Stocks,NIKKEI225,NaN,JPY,USD,2.365662e+06,21767.341884,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019-12-31,NaT,L,Stocks,SMI,NaN,CHF,USD,5.308470e+04,54849.902784,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019-12-31,2020-06-30,S,FX forward,FWD B:EUR S:USD 1.1205 06/30,NaN,USD,USD,-4.111564e+05,-411156.382050,...,NaN,NaN,NaT,FWDEUR0630,NaN,NaN,NaN,NaN,NaN,NaN
8,2019-12-31,NaT,L,Stocks,SPASX,NaN,AUD,USD,3.342050e+04,23493.007714,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019-12-31,NaT,L,Cash,USD,NaN,USD,USD,5.000000e+04,50000.000000,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Tenor (yrs),Euro,Switzerland,United Kingdom,Australia,United States,Japan
0,1,-0.00421,-0.00733,0.00716,0.00767,0.01654,-0.00085
1,2,-0.00391,-0.00706,0.00690,0.00783,0.01572,-0.00085
2,3,-0.00338,-0.00650,0.00715,0.00822,0.01565,-0.00090
3,4,-0.00285,-0.00598,0.00749,0.00916,0.01583,-0.00079
4,5,-0.00229,-0.00524,0.00778,0.00981,0.01606,-0.00069
5,6,-0.00164,-0.00469,0.00807,0.01054,0.01641,-0.00068
6,7,-0.00084,-0.00407,0.00836,0.01131,0.01678,-0.00051
7,8,-0.00018,-0.00342,0.00859,0.01203,0.01711,-0.00031
8,9,0.00047,-0.00281,0.00886,0.01271,0.01748,-0.00009
9,10,0.00113,-0.00199,0.00914,0.01334,0.01782,0.00014


,Asset type,Shock size,Description
0,Equity down,-0.200,Parallel
1,USD IR up,0.010,Parallel
2,USD IR down,-0.010,Parallel
3,EUR IR up,0.010,Parallel
4,EUR IR down,0.000,Parallel
5,GBP IR up,0.020,Parallel
6,GBP IR down,0.000,Parallel
7,CHF IR up,0.010,Parallel
8,CHF IR down,0.000,Parallel
9,AUD IR up,0.005,Parallel


,Corr between market factors,IR,FX,EQ,Spread
0,IR,1.00,0.25,0.00,0.25
1,FX,0.25,1.00,0.25,0.25
2,EQ,0.00,0.25,1.00,0.25
3,Spread,0.25,0.25,0.25,1.00


# Task 1

## Calculate partial risks of the individual assets

### stock 

In [26]:
PORTFOLIO['EQ down'][PORTFOLIO['Asset type'] == 'Stocks'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Stocks']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'Equity down'].iloc[0]
PORTFOLIO['FX up'][PORTFOLIO['Asset type'] == 'Stocks'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Stocks']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX up'].iloc[0]
PORTFOLIO['FX down'][PORTFOLIO['Asset type'] == 'Stocks'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Stocks']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX down'].iloc[0]
PORTFOLIO['IR up'][PORTFOLIO['Asset type'] == 'Stocks'] = 0.0
PORTFOLIO['IR down'][PORTFOLIO['Asset type'] == 'Stocks'] = 0.0
PORTFOLIO['Spread up'][PORTFOLIO['Asset type'] == 'Stocks'] = 0.0

display(PORTFOLIO[PORTFOLIO['Asset type'] == 'Stocks'])

,Valuation date,Maturity,Direction,Asset type,Asset ID,Option type,Quote ccy,Portfolio ccy,Mkt val qccy,Mkt val pccy,...,Cpn type,Cpn freq,Next cpn,Grouping ID 2 legs,FX up,FX down,IR up,IR down,EQ down,Spread up
4,2019-12-31,NaT,L,Stocks,SX5E,NaN,EUR,USD,112454.1,126252.302618,...,NaN,NaN,NaT,NaN,31563.075655,-31563.075655,0.0,0.0,-25250.460524,0.0
5,2019-12-31,NaT,L,Stocks,NIKKEI225,NaN,JPY,USD,2365662.0,21767.341884,...,NaN,NaN,NaT,NaN,5441.835471,-5441.835471,0.0,0.0,-4353.468377,0.0
6,2019-12-31,NaT,L,Stocks,SMI,NaN,CHF,USD,53084.7,54849.902784,...,NaN,NaN,NaT,NaN,13712.475696,-13712.475696,0.0,0.0,-10969.980557,0.0
8,2019-12-31,NaT,L,Stocks,SPASX,NaN,AUD,USD,33420.5,23493.007714,...,NaN,NaN,NaT,NaN,5873.251929,-5873.251929,0.0,0.0,-4698.601543,0.0
18,2019-12-31,NaT,L,Stocks,SP500,NaN,USD,USD,129231.2,129231.200000,...,NaN,NaN,NaT,NaN,32307.800000,-32307.800000,0.0,0.0,-25846.240000,0.0


### cash 

In [27]:
PORTFOLIO['EQ down'][PORTFOLIO['Asset type'] == 'Cash'] = 0.0
PORTFOLIO['FX up'][PORTFOLIO['Asset type'] == 'Cash'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Cash']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX up'].iloc[0]
PORTFOLIO['FX down'][PORTFOLIO['Asset type'] == 'Cash'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Cash']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX down'].iloc[0]
PORTFOLIO['FX up'][(PORTFOLIO['Asset type'] == 'Cash') & (PORTFOLIO['Quote ccy'] == 'USD')] = 0.0
PORTFOLIO['FX down'][(PORTFOLIO['Asset type'] == 'Cash') & (PORTFOLIO['Quote ccy'] == 'USD')] = 0.0
PORTFOLIO['IR up'][PORTFOLIO['Asset type'] == 'Cash'] = 0.0
PORTFOLIO['IR down'][PORTFOLIO['Asset type'] == 'Cash'] = 0.0
PORTFOLIO['Spread up'][PORTFOLIO['Asset type'] == 'Cash'] = 0.0

display(PORTFOLIO[PORTFOLIO['Asset type'] == 'Cash'])

,Valuation date,Maturity,Direction,Asset type,Asset ID,Option type,Quote ccy,Portfolio ccy,Mkt val qccy,Mkt val pccy,...,Cpn type,Cpn freq,Next cpn,Grouping ID 2 legs,FX up,FX down,IR up,IR down,EQ down,Spread up
9,2019-12-31,NaT,L,Cash,USD,NaN,USD,USD,50000.0,50000.000000,...,NaN,NaN,NaT,NaN,0.000000,0.000000,0.0,0.0,0.0,0.0
11,2019-12-31,NaT,L,Cash,JPY,NaN,JPY,USD,3000000.0,27604.123350,...,NaN,NaN,NaT,NaN,6901.030837,-6901.030837,0.0,0.0,0.0,0.0
14,2019-12-31,NaT,L,Cash,EUR,NaN,EUR,USD,50000.0,56135.037592,...,NaN,NaN,NaT,NaN,14033.759398,-14033.759398,0.0,0.0,0.0,0.0


### bond

In [28]:
PORTFOLIO['EQ down'][PORTFOLIO['Asset type'] == 'Bonds'] = 0.0
PORTFOLIO['FX up'][PORTFOLIO['Asset type'] == 'Bonds'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Bonds']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX up'].iloc[0]
PORTFOLIO['FX down'][PORTFOLIO['Asset type'] == 'Bonds'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Bonds']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX down'].iloc[0]
PORTFOLIO['FX up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'USD')] = 0.0
PORTFOLIO['FX down'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'USD')] = 0.0


PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'USD')] =\
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'USD')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR up'].iloc[0]
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'USD')] =\
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'USD')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR down'].iloc[0]

PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'EUR')] =\
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'EUR')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'EUR IR up'].iloc[0]
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'EUR')] =\
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'EUR')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'EUR IR down'].iloc[0]

PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'GBP')] =\
                        -PORTFOLIO['Notional/Quantity'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'GBP')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'GBP IR up'].iloc[0]
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'GBP')] =\
                        -PORTFOLIO['Notional/Quantity'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'GBP')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'GBP IR down'].iloc[0]

PORTFOLIO['Spread up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'USD')] = \
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'USD')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD Spread up'].iloc[0]

PORTFOLIO['Spread up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'EUR')] = \
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'EUR')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'EUR Spread up'].iloc[0]

PORTFOLIO['Spread up'][(PORTFOLIO['Asset type'] == 'Bonds') & (PORTFOLIO['Quote ccy'] == 'GBP')] = \
                        -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'Bonds')  & (PORTFOLIO['Quote ccy'] == 'GBP')]*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'GBP Spread up'].iloc[0]

display(PORTFOLIO[PORTFOLIO['Asset type'] == 'Bonds'])

,Valuation date,Maturity,Direction,Asset type,Asset ID,Option type,Quote ccy,Portfolio ccy,Mkt val qccy,Mkt val pccy,...,Cpn type,Cpn freq,Next cpn,Grouping ID 2 legs,FX up,FX down,IR up,IR down,EQ down,Spread up
1,2019-12-31,2025-04-15,L,Bonds,TNOTE,NaN,USD,USD,101057.0,101057.000000,...,Fixed,2.0,2020-04-15,NaN,0.000000,0.000000,-1010.570000,1010.57,0.0,-2021.140000
3,2019-12-31,2021-06-30,L,Bonds,UKST,NaN,GBP,USD,202114.0,268155.071870,...,Floating,4.0,2020-02-14,NaN,67038.767967,-67038.767967,-4000.000000,-0.00,0.0,-4022.326078
10,2019-12-31,2022-09-30,L,Bonds,DBUND,NaN,EUR,USD,106038.0,119048.942324,...,Fixed,1.0,2020-09-30,NaN,29762.235581,-29762.235581,-1190.489423,-0.00,0.0,-1190.489423


### FX forward

In [30]:
PORTFOLIO['EQ down'][PORTFOLIO['Asset type'] == 'FX forward'] = 0.0

PORTFOLIO['FX up'][PORTFOLIO['Asset type'] == 'FX forward'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'FX forward']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX up'].iloc[0]
PORTFOLIO['FX down'][PORTFOLIO['Asset type'] == 'FX forward'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'FX forward']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX down'].iloc[0]

PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'EUR')] =\
     -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'EUR')]*(PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR up'].iloc[0]-PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'EUR IR up'].iloc[0])
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'EUR')] =\
     -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'EUR')]*(PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR down'].iloc[0]-PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'EUR IR down'].iloc[0])

PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'JPY')] =\
     -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'JPY')]*(PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR up'].iloc[0])
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'JPY')] =\
     -PORTFOLIO['Mkt val pccy'][(PORTFOLIO['Asset type'] == 'FX forward') & (PORTFOLIO['Undl ccy'] == 'JPY')]*(PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR down'].iloc[0])

PORTFOLIO['Spread up'][PORTFOLIO['Asset type'] == 'FX forward'] = 0.0

display(PORTFOLIO[PORTFOLIO['Asset type'] == 'FX forward'])

,Valuation date,Maturity,Direction,Asset type,Asset ID,Option type,Quote ccy,Portfolio ccy,Mkt val qccy,Mkt val pccy,...,Cpn type,Cpn freq,Next cpn,Grouping ID 2 legs,FX up,FX down,IR up,IR down,EQ down,Spread up
2,2019-12-31,2020-09-30,S,FX forward,FWD B:USD S:JPY 106 09/30,NaN,JPY,USD,-1.000638e+07,-92072.403261,...,NaN,NaN,NaT,FWDJPY0930,-23018.100815,23018.100815,920.724033,-920.724033,0.0,0.0
7,2019-12-31,2020-06-30,S,FX forward,FWD B:EUR S:USD 1.1205 06/30,NaN,USD,USD,-4.111564e+05,-411156.382050,...,NaN,NaN,NaT,FWDEUR0630,-102789.095513,102789.095513,0.000000,-4111.563821,0.0,0.0
12,2019-12-31,2020-06-30,L,FX forward,FWD B:EUR S:USD 1.1205 06/30,NaN,EUR,USD,3.707789e+05,416273.693663,...,NaN,NaN,NaT,FWDEUR0630,104068.423416,-104068.423416,-0.000000,4162.736937,0.0,0.0
17,2019-12-31,2020-09-30,L,FX forward,FWD B:USD S:JPY 106 09/30,NaN,USD,USD,9.316934e+04,93169.339623,...,NaN,NaN,NaT,FWDJPY0930,23292.334906,-23292.334906,-931.693396,931.693396,0.0,0.0


### Equity futures

In [32]:
PORTFOLIO['EQ down'][PORTFOLIO['Asset type'] == 'Equity futures'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity futures']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'Equity down'].iloc[0]
PORTFOLIO['FX up'][PORTFOLIO['Asset type'] == 'Equity futures'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity futures']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX up'].iloc[0]
PORTFOLIO['FX down'][PORTFOLIO['Asset type'] == 'Equity futures'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity futures']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX down'].iloc[0]
PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'Equity futures') & (PORTFOLIO['Undl ccy'] == 'USD')] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity futures']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR up'].iloc[0]
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'Equity futures') & (PORTFOLIO['Undl ccy'] == 'USD') ] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity futures']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'USD IR down'].iloc[0]
PORTFOLIO['Spread up'][PORTFOLIO['Asset type'] == 'Equity futures'] = 0.0

display(PORTFOLIO[PORTFOLIO['Asset type'] == 'Equity futures'])

,Valuation date,Maturity,Direction,Asset type,Asset ID,Option type,Quote ccy,Portfolio ccy,Mkt val qccy,Mkt val pccy,...,Cpn type,Cpn freq,Next cpn,Grouping ID 2 legs,FX up,FX down,IR up,IR down,EQ down,Spread up
15,2019-12-31,2020-09-30,S,Equity futures,E-MINI U,NaN,USD,USD,-17946.982900,-17946.982900,...,NaN,NaN,NaT,NaN,-4486.745725,4486.745725,-179.469829,179.469829,3589.396580,0.0
16,2019-12-31,2020-09-30,S,Equity futures,NIKKEI 225 U,NaN,JPY,USD,-266373.438945,-2451.001755,...,NaN,NaN,NaT,NaN,-612.750439,612.750439,NaN,NaN,490.200351,0.0


### Equity options

In [ ]:
PORTFOLIO['EQ down'][PORTFOLIO['Asset type'] == 'Equity options'] = 
PORTFOLIO['FX up'][PORTFOLIO['Asset type'] == 'Equity options'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity options']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX up'].iloc[0]
PORTFOLIO['FX down'][PORTFOLIO['Asset type'] == 'Equity options'] = PORTFOLIO['Mkt val pccy'][PORTFOLIO['Asset type'] == 'Equity options']*PSHOCKS['Shock size'][PSHOCKS['Asset type'] == 'FX down'].iloc[0]
PORTFOLIO['FX up'][(PORTFOLIO['Asset type'] == 'Equity options') & (PORTFOLIO['Undl ccy'] == 'USD')] = 0.0
PORTFOLIO['FX down'][(PORTFOLIO['Asset type'] == 'Equity options') & (PORTFOLIO['Undl ccy'] == 'USD')] = 0.0
PORTFOLIO['IR up'][(PORTFOLIO['Asset type'] == 'Equity options') & (PORTFOLIO['Undl ccy'] == 'USD')] = 
PORTFOLIO['IR down'][(PORTFOLIO['Asset type'] == 'Equity options') & (PORTFOLIO['Undl ccy'] == 'USD') ] = 
PORTFOLIO['Spread up'][PORTFOLIO['Asset type'] == 'Equity options'] = 0.0

display(PORTFOLIO[PORTFOLIO['Asset type'] == 'Equity options'])

# Task 2